In [267]:
from scipy.special import erf as norm_erf
from math import pi,sqrt
from torch import erf as norm_erf
import torch
from torch import tensor as tt
from scipy.optimize import bisect
import numpy as np
import pandas as pd

In [116]:
norm_factor = sqrt(pi)/2
erf = lambda z: norm_erf(z)*norm_factor

In [117]:
def zero_one_gaussian_integral(mu,sigma):
    upper_dist = 1-mu
    lower_dist = mu
    z_up = upper_dist/sigma
    z_low = lower_dist/sigma
    return sigma*(erf(z_up)+erf(z_low))

def zero_one_gaussian_square_integral(mu,sigma):
    return zero_one_gaussian_integral(mu,sigma/sqrt(2))

def zero_one_gaussian_variance(mu,sigma):
    return zero_one_gaussian_square_integral(mu,sigma) - (zero_one_gaussian_integral(mu,sigma))**2

In [339]:
def hypercube_integral(mu,sigma):
    return torch.prod(zero_one_gaussian_integral(mu,sigma))

def hypercube_variance(mu,sigma):
    return torch.prod(zero_one_gaussian_square_integral(mu,sigma)) - hypercube_integral(mu,sigma)**2

def hypercube_rel_variance(mu,sigma):
    return hypercube_variance(mu,sigma)/hypercube_integral(mu,sigma)/hypercube_integral(mu,sigma)

def hypercube_camel_integral(d,sigma):
    mu025 = tt([0.25]*d)
    mu075 = tt([0.75]*d)
    return hypercube_integral(mu025,sigma)+hypercube_integral(mu075,sigma)

def hypercube_camel_variance(d,sigma):
    mu025 = tt([0.25]*d)
    mu075 = tt([0.75]*d)
    return hypercube_variance(mu025,sigma)+hypercube_variance(mu075,sigma)

def hypercube_camel_rel_variance(d,sigma):
    return hypercube_camel_variance(d,sigma)/hypercube_camel_integral(d,sigma)/hypercube_camel_integral(d,sigma)

In [340]:
gaussian_sigma_std_map = dict()
camel_sigma_std_map = dict()
for sig in [.001,.01,.1,.3,.5,.7,1.]:
    gauss = hypercube_rel_variance(tt([0.5]),tt(sig)).sqrt().item()
    camel = hypercube_camel_rel_variance(1,tt(sig)).sqrt().item()
    print("gauss",sig, gauss)
    print("camel",sig, camel)
    gaussian_sigma_std_map[sig] = gauss
    camel_sigma_std_map[sig] = camel
    print("")

gauss 0.001 19.948490142822266
camel 0.001 14.105712890625

gauss 0.01 6.236523628234863
camel 0.01 4.40988826751709

gauss 0.1 1.728994607925415
camel 0.1 1.2229156494140625

gauss 0.3 0.615635335445404
camel 0.3 0.562710702419281

gauss 0.5 0.269128680229187
camel 0.5 0.32639604806900024

gauss 0.7 0.14459455013275146
camel 0.7 0.1965818554162979

gauss 1.0 0.07274238765239716
camel 1.0 0.10544660687446594



In [341]:
def find_sigma(mu, rel_std_target,d=1):
    rel_var_target = tt(rel_std_target, dtype=torch.float)**2
    if isinstance(mu,float):
        mu = tt([mu]*d)
    def f(sig):
        return (hypercube_rel_variance(mu,sig) - rel_var_target).item()
    sig = bisect(f,1.e-2,10)
    return sig

In [342]:
sig = find_sigma(0.5, 100. ,d=16)
print(sig)

0.2250947132709564


In [343]:
def find_sigma_camel(d, rel_std_target):
    rel_var_target = tt(rel_std_target, dtype=torch.float)**2
    def f(sig):
        return (hypercube_camel_rel_variance(d,sig) - rel_var_target).item()
    sig = bisect(f,1.e-2,3)
    return sig

In [344]:
sig = find_sigma_camel(16, 100.)
print(sig)

0.24595979601194357


In [368]:
gaussian_sigma_mapping = pd.DataFrame()
for sig,std_tgt in gaussian_sigma_std_map.items():
    for d in [2,4,8,16]:
        sigma = find_sigma(0.5,std_tgt,d)
        gaussian_sigma_mapping = gaussian_sigma_mapping.append(
        {
            "d":d,
            "sigma_1d":sig,
            "sigma_gauss":sigma,
            "relative_std_gauss":std_tgt
        },
            ignore_index=True
        )
        

In [369]:
camel_sigma_mapping = pd.DataFrame()
for sig,std_tgt in camel_sigma_std_map.items():
    for d in [2,4,8,16]:
        sigma = find_sigma_camel(d,std_tgt)
        camel_sigma_mapping = camel_sigma_mapping.append(
        {
            "d":d,
            "sigma_1d":sig,
            "sigma_camel":sigma,
            "relative_std_camel":std_tgt
        },
            ignore_index=True
        )

In [370]:
def find_sigma_camel_32(rel_std_target):
    d=32
    rel_var_target = tt(rel_std_target, dtype=torch.float)**2
    def f(sig):
        return (hypercube_camel_rel_variance(d,sig) - rel_var_target).item()
    sig = bisect(f,1.e-1,10)
    return sig
def find_sigma_32(rel_std_target):
    d=32
    mu=0.5
    rel_var_target = tt(rel_std_target, dtype=torch.float)**2
    if isinstance(mu,float):
        mu = tt([mu]*d)
    def f(sig):
        return (hypercube_rel_variance(mu,sig) - rel_var_target).item()
    sig = bisect(f,1.e-1,10)
    return sig

In [371]:
d = 32
for sig,std_tgt in gaussian_sigma_std_map.items():
    sigma = find_sigma_32(std_tgt)
    gaussian_sigma_mapping = gaussian_sigma_mapping.append(
    {
        "d":d,
        "sigma_1d":sig,
        "sigma_gauss":sigma,
        "relative_std_gauss":std_tgt
    },
        ignore_index=True
    )
for sig,std_tgt in camel_sigma_std_map.items():
    sigma = find_sigma_camel_32(std_tgt)
    camel_sigma_mapping = camel_sigma_mapping.append(
    {
        "d":d,
        "sigma_1d":sig,
        "sigma_camel":sigma,
        "relative_std_camel":std_tgt
    },
        ignore_index=True
    )        

In [373]:
d_agnostic_setups = camel_sigma_mapping.set_index(["d","sigma_1d"]).join(
gaussian_sigma_mapping.set_index(["d","sigma_1d"])).reset_index()

In [374]:
d_agnostic_setups

,d,sigma_1d,relative_std_camel,sigma_camel,relative_std_gauss,sigma_gauss
0,2.0,0.001,14.105713,0.019974,19.948490,0.019974
1,4.0,0.001,14.105713,0.089272,19.948490,0.089265
2,8.0,0.001,14.105713,0.203943,19.948490,0.188778
3,16.0,0.001,14.105713,0.361255,19.948490,0.280880
4,2.0,0.010,4.409888,0.063162,6.236524,0.063162
5,4.0,0.010,4.409888,0.163530,6.236524,0.158741
6,8.0,0.010,4.409888,0.314475,6.236524,0.254390
7,16.0,0.010,4.409888,0.463898,6.236524,0.341485
8,2.0,0.100,1.222916,0.220838,1.728995,0.199897
9,4.0,0.100,1.222916,0.378421,1.728995,0.290792


In [375]:
d_agnostic_setups.to_csv("gaussian_camel_integrands.csv")

In [376]:
ls

Untitled.ipynb                 gaussian_camel_integrands.csv
